# Q&A with the SEC Search Agent

This notebook demonstrates how to interact with the SEC EDGAR data API using Amazon Bedrock Agents.

## 0. Testing

In [ ]:
!python3 -m pip install --upgrade -q botocore boto3 awscli sec-edgar-api rapidfuzz

In [1]:
from typing import Dict, Optional
import json
from pathlib import Path
from rapidfuzz import process, fuzz, utils


def get_cik(query: str, data_file: Path) -> Optional[Dict]:
    """
    Look up the SEC Central Index Key (CIK) for a given company name using fuzzy matching.

    Args:
        query (str): The company name to search for
        data_file (Path): Path to the JSON file containing company data

    Returns:
        Optional[Dict]: Company information dictionary if found, None if no match or error

    Raises:
        FileNotFoundError: If the data_file doesn't exist
        json.JSONDecodeError: If the data_file contains invalid JSON
    """
    try:
        with open(data_file, "r", encoding="utf-8") as f:
            company_tickers = json.load(f)

        if not company_tickers:
            return None

        choices = [company.get("title", "") for company in company_tickers.values()]

        match = process.extractOne(
            query,
            choices,
            scorer=fuzz.WRatio,
            processor=utils.default_process,
            score_cutoff=80,  # Minimum similarity score threshold
        )

        return list(company_tickers.values())[match[2]] if match else None

    except (FileNotFoundError, json.JSONDecodeError) as e:
        # Consider logging the error here
        raise e

In [3]:
from sec_edgar_api import EdgarClient
edgar = EdgarClient(user_agent="Amazon Web Services bloyal@amazon.com")
cik = get_cik("pfizer", "company_tickers.json").get("cik_str", "")

edgar.get_submissions(cik)
# edgar.get_company_concept(cik=cik, taxonomy="us-gaap", tag="AccountsPayableCurrent")


{'cik': '0000078003',
 'entityType': 'operating',
 'sic': '2834',
 'sicDescription': 'Pharmaceutical Preparations',
 'ownerOrg': '03 Life Sciences',
 'insiderTransactionForOwnerExists': 1,
 'insiderTransactionForIssuerExists': 1,
 'name': 'PFIZER INC',
 'tickers': ['PFE'],
 'exchanges': ['NYSE'],
 'ein': '135315170',
 'lei': None,
 'description': '',
 'website': '',
 'investorWebsite': '',
 'category': 'Large accelerated filer',
 'fiscalYearEnd': '1231',
 'stateOfIncorporation': 'DE',
 'stateOfIncorporationDescription': 'DE',
 'addresses': {'mailing': {'street1': '66 HUDSON BOULEVARD EAST',
   'street2': None,
   'city': 'NEW YORK',
   'stateOrCountry': 'NY',
   'zipCode': '10001-2192',
   'stateOrCountryDescription': 'NY',
   'isForeignLocation': 0,
   'foreignStateTerritory': None,
   'country': None,
   'countryCode': None},
  'business': {'street1': '66 HUDSON BOULEVARD EAST',
   'street2': None,
   'city': 'NEW YORK',
   'stateOrCountry': 'NY',
   'zipCode': '10001-2192',
   'stat

In [ ]:
import json

edgar_response = edgar.get_company_facts(cik=cik)

facts = {
    k: v["description"]
    for _, tv in edgar_response["facts"].items()
    for k, v in tv.items()
}

for i in facts.items():
    print(i)

with open("facts.json", "w") as f:
    json.dump(facts, f)


In [ ]:
for k,v in facts['facts']['us-gaap'].items():
    print(k)
    print(v['label'])
    print(v['description'])
    print()

In [ ]:
for i in facts['facts']['srt'].keys():
    print(i)

In [ ]:
[
    data
    for data in edgar.get_frames(
        taxonomy="us-gaap",
        tag="AccountsPayableCurrent",
        unit="USD",
        year="2024",
        quarter=4,
    ).get("data") if data.get("cik") == cik
]